In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from models.regression.MLPRegressionCustom import MLP
from models.metrics.classification import  Recall, F1Score, Precision
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

In [2]:
df = np.genfromtxt('./vowel.csv', delimiter=',')
X = df[:, :-1]
y = df[:, -1].reshape(-1, 1)

X = MinMaxScaler().fit_transform(X)
enc = OneHotEncoder()
enc.fit(y)
y = enc.transform(y).toarray()

### Train Test Split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)
_X_train, X_val, _y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [4]:
neuronios = [3, 5, 10, 15]
learning_rate = [0.1, 0.15, 0.25, 0.4]
momentum = [0.3, 0.4, 0.9]

result = []

elected = None

for n in neuronios:
    for l in learning_rate:
        for m in momentum:
            net = MLP(_X_train, _y_train, n, eta=l, niterations=200, momentum=m, outtype="logistic", metric=Precision(), encoder=enc)
            net.fit(_X_train, _y_train, X_val, y_val)
            result.append([net, np.min(net.errors_valid)])
            print("Neuronios: {} Learning Rate: {}, Momentum: {}. Score: {}".format(n, l, m, np.min(net.errors_train)))

elected = result[np.argmin(np.array(result)[:, 1])][0]

Iteration:  0  Error:  0.0
Iteration:  100  Error:  0.0


ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
net = MLP(_X_train, _y_train, elected.nhidden, eta=elected.eta, niterations=201, momentum=elected.momentum, outtype="logistic", metric=Precision)
net.fit(_X_train, _y_train, X_test, y_test)
plt.plot(net.errors_valid)
plt.plot(net.errors_train)
plt.show()